In [2]:
import pandas as pd
df=pd.read_csv('parkinsons.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              195 non-null    object 
 1   MDVP:Fo(Hz)       195 non-null    float64
 2   MDVP:Fhi(Hz)      195 non-null    float64
 3   MDVP:Flo(Hz)      195 non-null    float64
 4   MDVP:Jitter(%)    195 non-null    float64
 5   MDVP:Jitter(Abs)  195 non-null    float64
 6   MDVP:RAP          195 non-null    float64
 7   MDVP:PPQ          195 non-null    float64
 8   Jitter:DDP        195 non-null    float64
 9   MDVP:Shimmer      195 non-null    float64
 10  MDVP:Shimmer(dB)  195 non-null    float64
 11  Shimmer:APQ3      195 non-null    float64
 12  Shimmer:APQ5      195 non-null    float64
 13  MDVP:APQ          195 non-null    float64
 14  Shimmer:DDA       195 non-null    float64
 15  NHR               195 non-null    float64
 16  HNR               195 non-null    float64
 1

In [4]:
df.head()

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [5]:
df.isnull().sum()

name                0
MDVP:Fo(Hz)         0
MDVP:Fhi(Hz)        0
MDVP:Flo(Hz)        0
MDVP:Jitter(%)      0
MDVP:Jitter(Abs)    0
MDVP:RAP            0
MDVP:PPQ            0
Jitter:DDP          0
MDVP:Shimmer        0
MDVP:Shimmer(dB)    0
Shimmer:APQ3        0
Shimmer:APQ5        0
MDVP:APQ            0
Shimmer:DDA         0
NHR                 0
HNR                 0
status              0
RPDE                0
DFA                 0
spread1             0
spread2             0
D2                  0
PPE                 0
dtype: int64

seperate features and target


In [6]:
x=df.drop(columns=['name','status'],axis=1)
y=df['status']


In [7]:
import joblib
from sklearn.preprocessing import StandardScaler

# suppose df is your dataset
num_cols = df.select_dtypes(include=['int64','float64']).columns  

scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

# save (dump) the scaler
joblib.dump(scaler, "parkinsons_scaler.pkl")
print("Scaler saved successfully!")


Scaler saved successfully!


train test

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2)


random forest classifier

In [9]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(x_train,y_train)
y_pred=rf.predict(x_test)
accuracy=rf.score(x_test,y_test)
print(accuracy)

0.8205128205128205


Evaluate Using Other Metrics

Accuracy alone isn’t enough, especially for medical datasets. Check:

Precision → How many predicted positives are actually positive

Recall (Sensitivity) → How many actual positives did the model detect

F1-Score → Balance between Precision and Recall

ROC-AUC → How well the model separates classes

In [10]:
# Predict class labels (0 or 1)
y_pred = rf.predict(x_test)

# Predict probabilities for positive class (e.g., CKD=1 or Parkinson's=1)
y_prob = rf.predict_proba(x_test)[:,1]  # [:,1] gets probability of class 1


In [11]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_prob)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
print("ROC-AUC:", roc_auc)


Precision: 0.9285714285714286
Recall: 0.8387096774193549
F1-Score: 0.8813559322033899
ROC-AUC: 0.9092741935483871


Precision = 0.90

Out of all the samples predicted as Parkinson’s, 90% were actually Parkinson’s.

High precision → few false positives (healthy people wrongly predicted as Parkinson’s).

2️⃣ Recall = 0.871

Out of all actual Parkinson’s patients, the model correctly identified 87.1%.

High recall → few false negatives (Parkinson’s patients missed by the model).

3️⃣ F1-Score = 0.885

Harmonic mean of Precision and Recall:

Precision+Recall
Precision×Recall
	​


F1 = 0.885 → good balance between Precision and Recall.

4️⃣ ROC-AUC = 0.903

ROC-AUC measures how well the model separates positive vs negative classes.

0.90 → model can distinguish Parkinson’s vs healthy 90% of the time.

In [12]:
import joblib
joblib.dump(rf, 'parkinsons_model.pkl')

['parkinsons_model.pkl']